In [ ]:
from tkinter import *
from PIL import Image, ImageTk

class Player:
    def __init__(self, player_id, team_id, ball_possession, ball_received, x, y,upd):
        self.upd=upd
        self.player_id = player_id
        self.team_id = team_id
        self.ball_possession = ball_possession
        self.ball_received = ball_received
        self.x = x
        self.y = y

class ImageWindow:
    def __init__(self, root): 
        self.i=1
        self.root = root
        self.root.title("Image Window")
        self.count=0
        # Create canvas widget and scrollbars
        self.canvas = Canvas(self.root)
        self.scrollbarx = Scrollbar(self.root, orient=HORIZONTAL, command=self.canvas.xview)
        self.scrollbary = Scrollbar(self.root, orient=VERTICAL, command=self.canvas.yview)
        self.canvas.configure(xscrollcommand=self.scrollbarx.set, yscrollcommand=self.scrollbary.set)
        self.canvas.pack(side=LEFT, fill=BOTH, expand=True)
        self.scrollbarx.pack(side=BOTTOM, fill=X)
        self.scrollbary.pack(side=RIGHT, fill=Y)
        
        # Load image and display on canvas
        self.image = Image.open("FIFA_gnd.jpg")
        self.img_width, self.img_height = self.image.size
        self.img = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, image=self.img, anchor="nw")
        self.canvas.config(scrollregion=(0,0,self.img_width,self.img_height))
        
        # Create slider widget for adjusting image size
        self.scale = Scale(self.root, from_=0.1, to=3.0, resolution=0.1, orient=HORIZONTAL, command=self.scale_image)
        self.scale.set(1.0)
        self.scale.pack(side=TOP, anchor=W, padx=10)
        
        # Create entry widget for data name
        # self.data_name_label = Label(self.root, text="Image Name:")
        # self.data_name_label.pack(side=LEFT, padx=10, pady=10)
        # self.data_name_entry = Entry(self.root)
        # self.data_name_entry.pack(side=TOP, padx=10, pady=10)
        label_frame = Frame(self.root)
        label_frame.pack(side=TOP, padx=10, pady=10)

        data_label = Label(label_frame, text="Image Name:")
        data_label.pack(side=LEFT)

        self.data_name_entry = Entry(label_frame)
        self.data_name_entry.pack(side=LEFT)
        # Bind mouse clicks to canvas
        self.canvas.bind("<Button-1>", self.add_point)
        
        # Create list of player objects
        self.player_list = []
        
        # Create dictionary for team colors
        self.team_colors = {0: "black", 1: "blue", 2: "red", 3: "yellow", 4: "green", 5: "pink", 6: "white"}
        

        # Create checkbox for showing labels
        self.show_labels_var = BooleanVar()
        self.show_labels_var.set(True)
        self.show_labels_checkbutton = Checkbutton(self.root, text="Show Labels", variable=self.show_labels_var, command=self.show_labels)
        self.show_labels_checkbutton.pack(side=TOP, padx=10, pady=10)
        
        # Create Clear and Submit buttons
        self.clear_button = Button(self.root, text="Clear", command=self.clear_data)
        self.clear_button.pack(side=LEFT, padx=10, pady=10)
        self.submit_button = Button(self.root, text="Submit", command=self.submit_data)
        self.submit_button.pack(side=RIGHT, padx=10, pady=10)
        
        label_frame = Frame(self.root)
        label_frame.pack(side=TOP, padx=10, pady=10)
        self.tid_var = IntVar(value=0)
        team_label = Label(label_frame, text="Team ID:")
        team_label.pack(side=TOP,  padx=15, pady=15)
        ti_0 = Radiobutton(label_frame, text="left", variable=self.tid_var, value=0)
        ti_1 = Radiobutton(label_frame, text="right",variable=self.tid_var, value=1)
        ti_2 = Radiobutton(label_frame, text="reff", variable=self.tid_var, value=2)
        ti_3 = Radiobutton(label_frame, text="goal", variable=self.tid_var, value=3)
        
        
        ti_0.pack(side=LEFT, padx=5, pady=5)
        ti_1.pack(side=LEFT, padx=5, pady=5)
        ti_2.pack(side=LEFT, padx=5, pady=5)
        ti_3.pack(side=LEFT, padx=5, pady=5)
        
    def scale_image(self, val):
        """Scale the image on the canvas"""
        scale_val = float(val)
        new_width = int(self.img_width * scale_val)
        new_height = int(self.img_height * scale_val)
        resized_img = self.image.resize((new_width, new_height))
        self.img = ImageTk.PhotoImage(resized_img)
                # Update canvas with resized image
        self.canvas.itemconfig(1, image=self.img)
        

    def add_point(self, event):
        """Add a new point object to the canvas and player list"""
        x, y = event.x, event.y
        x_scaled = round((x-105) * 0.115,2)
        y_scaled = round((690-y) * 0.115,2)
        

        # Check if click is too close to existing points
        too_close = False
        for item in self.canvas.find_withtag("point"):
            x1, y1, x2, y2 = self.canvas.coords(item)
            distance = ((x - (x1 + x2) / 2) ** 2 + (y - (y1 + y2) / 2) ** 2) ** 0.5
            if distance < 10:
                too_close = True
                break
            
        if too_close:
            return
        self.count +=1
        # Create new point object and add to canvas and player list
        team_id = self.tid_var.get()#len(self.team_colors) % 6
        if int(team_id) == 1:
            oval_color = "red"
        else:
            oval_color = "blue"
        oval_id = self.canvas.create_oval(x-5, y-5, x+5, y+5, fill=oval_color, tags=("point", f"point{len(self.player_list) + 1}"))
        player = Player(self.count, team_id, 0, 0, x_scaled, y_scaled,self.count)
        self.player_list.append(player)
        self.canvas.tag_bind(oval_id, "<Button-1>", lambda event, upd=player.upd: self.edit_player(upd))
        
        # Draw circles if ball possession or ball received is set to 1
        circle_id = self.canvas.create_oval(x-10, y-10, x+10, y+10, fill="",outline="green", tags=("circle", f"circle{player.upd}"))
  
        # Update labels if checkbox is checked
        self.show_labels()
        

    def edit_player(self, upd):
        """Display a popup window to edit player attributes"""
        player = self.get_player_by_id(upd)

        # Create popup window
        top = Toplevel()
        top.title(f"Edit Player {upd}")
        top.geometry("400x250")

        # Create labels and text fields for player attributes
        id_label = Label(top, text="Player ID:")
        id_label.grid(row=0, column=0, padx=5, pady=5)
        id_field = Entry(top, width=10)
        id_field.grid(row=0, column=1, padx=5, pady=5)
        id_field.insert(0, str(player.player_id))

        tid_var = IntVar(value=player.team_id)
        team_label = Label(top, text="Team ID:")
        team_label.grid(row=1, column=0, padx=5, pady=5)
        ti_0 = Radiobutton(top, text="left", variable=tid_var, value=0)
        ti_1 = Radiobutton(top, text="right",variable=tid_var, value=1)
        ti_2 = Radiobutton(top, text="reff", variable=tid_var, value=2)
        ti_3 = Radiobutton(top, text="goal", variable=tid_var, value=3)
        
        ti_0.grid(row=1, column=1, padx=5, pady=5)
        ti_1.grid(row=1, column=2, padx=5, pady=5)
        ti_2.grid(row=1, column=3, padx=5, pady=5)
        ti_3.grid(row=1, column=4, padx=5, pady=5)
        
        bp_var = IntVar(value=player.ball_possession)
        br_var = IntVar(value=player.ball_received)

        bp_label = Label(top, text="Ball Possession:")
        bp_label.grid(row=2, column=0, padx=5, pady=5)
        bp_0 = Radiobutton(top, text="0", variable=bp_var, value=0)
        bp_1 = Radiobutton(top, text="1", variable=bp_var, value=1)
        bp_0.grid(row=2, column=1, padx=5, pady=5)
        bp_1.grid(row=2, column=2, padx=5, pady=5)

        br_label = Label(top, text="Ball Received:")
        br_label.grid(row=3, column=0, padx=5, pady=5)
        br_0 = Radiobutton(top, text="0", variable=br_var, value=0)
        br_1 = Radiobutton(top, text="1", variable=br_var, value=1)
        br_0.grid(row=3, column=1, padx=5, pady=5)
        br_1.grid(row=3, column=2, padx=5, pady=5)
        
        x=player.x
        y=player.y

        x_label = Label(top, text="X:")
        x_label.grid(row=4, column=0, padx=5, pady=5)
        x_field = Entry(top, width=10)
        x_field.grid(row=4, column=1, padx=5, pady=5)
        x_field.insert(0, str(player.x))

        y_label = Label(top, text="Y:")
        y_label.grid(row=5, column=0, padx=5, pady=5)
        y_field = Entry(top, width=10)
        y_field.grid(row=5, column=1, padx=5, pady=5)
        y_field.insert(0, str(player.y))

        # Create buttons to submit or remove player
        submit_button = Button(top, text="Submit", command=lambda: self.update_player(upd, id_field.get(), tid_var.get(), bp_var.get(), br_var.get(), x,y, top))
        submit_button.grid(row=6, column=1, padx=5, pady=10)
        remove_button = Button(top, text="Remove", command=lambda: self.remove_player(upd, top))
        remove_button.grid(row=6, column=2, padx=5, pady=10)

    def inv(self,scaled_x,scaled_y):
        x = float(scaled_x) / 0.115 + 105
        y = 690 - float(scaled_y) / 0.115
        
        return x,y
        
    def update_player(self, upd, new_id, new_team, new_bp, new_br, new_x, new_y, top):
        """Update the attributes of a player object"""
        player = self.get_player_by_id(upd)
        # x = float(new_x) / 0.115 + 105
        # y = 690 - float(new_y) / 0.115
        x,y=self.inv(new_x,new_y)
        # Handle ball possession
        if int(new_bp) == 1:
            # Set all other players' ball possession to 0
            for other_player in self.player_list:
                if other_player != player and other_player.ball_possession == 1:
                    other_player.ball_possession = 0
                    #other_player.ball_received = 0
                    other_circle_id = self.canvas.find_withtag(f"circle{other_player.upd}")
                    self.canvas.itemconfig(other_circle_id, outline="green")
            # Set current player's ball possession to 1
            player.ball_possession = 1
            player.ball_received = 0
            circle_id = self.canvas.find_withtag(f"circle{player.upd}")
            #print("white",circle_id)
            self.canvas.itemconfig(circle_id, outline="white")
        else:
            player.ball_possession = 0


        # Handle ball received
        if int(new_br) == 1:
            # Set all other players' ball received to 0
            for other_player in self.player_list:
                if other_player != player and other_player.ball_received == 1:
                    other_player.ball_received = 0
                    other_circle_id = self.canvas.find_withtag(f"circle{other_player.upd}")
                    self.canvas.itemconfig(other_circle_id, outline="green")
            # Set current player's ball received to 1
            player.ball_received = 1
            player.ball_possession = 0
            circle_id = self.canvas.find_withtag(f"circle{player.upd}")
            self.canvas.itemconfig(circle_id, outline="black")
        else:
            player.ball_received = 0

        # # Handle both ball possession and ball received
        if player.ball_possession == 1 and player.ball_received == 1:
            player.ball_received = 0
            circle_id = self.canvas.find_withtag(f"circle{player.upd}")
            self.canvas.itemconfig(circle_id, outline="white")

        if player.ball_possession == 0 and player.ball_received == 0:
            player.ball_received = 0
            circle_id = self.canvas.find_withtag(f"circle{player.upd}")
            self.canvas.itemconfig(circle_id, outline="green")
            
        team_id = int(new_team)
        if team_id == 0:
            oval_color = "blue"
        elif team_id == 1:
            oval_color = "red"
        elif team_id == 2:
            oval_color = "yellow"
        elif team_id == 3:
            oval_color = "green"

        oval_id = self.canvas.find_withtag(f"point{player.upd}")
        self.canvas.itemconfig(oval_id, fill=oval_color)

        player.player_id = int(new_id)
        player.team_id = int(new_team)
        player.x = float(new_x)
        player.y = float(new_y)
        
        
        possessing_player = None
        receiving_player = None
        for other_player in self.player_list:
            if other_player.ball_possession == 1:
                possessing_player = other_player
            if other_player.ball_received == 1:
                receiving_player = other_player

        if possessing_player is not None and receiving_player is not None:
            start_x,start_y = self.inv(possessing_player.x,possessing_player.y)
            end_x,end_y = self.inv(receiving_player.x,receiving_player.y)
            self.canvas.delete("arrow")
            arrow_id = self.canvas.create_line(start_x, start_y, end_x, end_y, arrow="last", tags=("arrow",))

        # Update labels
        self.show_labels()

        # Close popup window
        top.destroy()


    def remove_player(self, upd, top):
        """Remove a player object and its oval from the canvas and close the pop-up window"""
        self.canvas.delete(f"point{upd}",f"circle{upd}",)
        self.player_list.pop(upd - 1)
        top.destroy()
        
        # Update labels if checkbox is checked
        self.show_labels()
    
    def get_player_by_id(self, upd):
        for player in self.player_list:
            if player.upd == upd:
                return player

    def show_labels(self):
        """Display the attributes of all player objects next to their respective ovals"""
        if self.show_labels_var.get():
            for player in self.player_list:
                oval_id = self.canvas.find_withtag(f"point{player.upd}")[0]
            
                oval_x, oval_y = self.canvas.coords(oval_id)[0], self.canvas.coords(oval_id)[1]
                label_text = f"Player ID: {player.player_id}\nTeam ID: {player.team_id}\nBall Possession: {player.ball_possession}\nBall Received: {player.ball_received}\nX: {player.x}\nY: {player.y}"
                label_id = self.canvas.create_text(oval_x + 10, oval_y - 10, text=label_text, anchor="nw", tags=("label", f"label{player.player_id}"))
        else:
            for item in self.canvas.find_withtag("label"):
                self.canvas.delete(item)
            
            
    def clear_data(self):
        """Clear all player objects and ovals from the canvas"""
        self.canvas.delete("point", "label","circle","arrow")
        self.count=0
        self.player_list.clear()
        
    def submit_data(self):
        """Append the player data to a CSV file"""
        file_name = "Template.csv"
        data_name = "simple{}".format(self.i)#self.data_name_entry.get()
        self.i+=1
        print(self.i)
        with open(file_name, "a") as f:
            for player in self.player_list:
                data_row = f"{data_name},{player.player_id},{player.team_id},{player.x},{player.y},{player.ball_possession},{player.ball_received}\n"
                f.write(data_row)
        #self.clear_data()
        self.show_labels()

root = Tk()
app = ImageWindow(root)
root.mainloop()